# Install Python packages

In [ ]:
%pip -q install -U dotenv azure-ai-ml azure-identity azureml-mlflow mlflow

# Load environment variables from a .env file
secret 노출을 피하고 notebook 들간의 일관된 환경변수를 설정하기 위해 `dotenv` 을 이용한다.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

AZURE_AML_SUBSCRIPTION_ID = os.getenv("AZURE_AML_SUBSCRIPTION_ID")
AZURE_AML_RESOURCE_GROUP = os.getenv("AZURE_AML_RESOURCE_GROUP")
AZURE_AML_WORKSPACE = os.getenv("AZURE_AML_WORKSPACE")
AZURE_AML_CLUSTER = os.getenv("AZURE_AML_CLUSTER")

# Create a Azure Machine Learning cilent
Azure Machine Learning 의 Client 객체인 `MLClient` 을 생성한다. 본 예제는 Azure CLI 로그인 Credential 을 사용하고 있다. 터미널에서 `az login` 을 정상적으로 완료하여야 한다. `az` 명령어가 설치되어 있지 않다면 [Azure CLI 설치하는 방법](https://learn.microsoft.com/ko-kr/cli/azure/install-azure-cli?view=azure-cli-latest) 을 참고한다.

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

ml_client = MLClient(
    DefaultAzureCredential(),
    AZURE_AML_SUBSCRIPTION_ID,
    AZURE_AML_RESOURCE_GROUP,
    AZURE_AML_WORKSPACE,
)

# Workflow pipeline
AML 은 `airflow` 와 같은 task management framework 와 같이 DAG 기반의 pipeline 을 지원한다. 사용자는 component 간의 재사용성을 높이며 견고한 개발 파이프라인을 구축해 나갈 수 있다.
예제 시나리오는, 일반적인 ML Pipeline 에서의 "모델 학습" - "평가 (추론 포함)" 으로 구성된다.

## Load workflow components
Pipeline 에 사용할 components 를 로드한다.

In [ ]:
from azure.ai.ml import MLClient, Input
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component

train_model = load_component(source="./resources/pipeline-from-yaml/train_model.yml")
score_data = load_component(source="./resources/pipeline-from-yaml/score_data.yml")
eval_model = load_component(source="./resources/pipeline-from-yaml/eval_model.yml")
print(train_model)

## Create a pipeline and submit it
AML 의 `pipeline` 기능을 통해 job 을 생성하여 AML 에 제출한다.

In [ ]:
# Construct pipeline
@pipeline()
def pipeline_with_components_from_yaml(
    training_input,
    test_input,
    training_max_epochs=20,
    training_learning_rate=1.8,
    learning_rate_schedule="time-based",
):
    train_mdl = train_model(
        training_data=training_input,
        max_epochs=training_max_epochs,
        learning_rate=training_learning_rate,
        learning_rate_schedule=learning_rate_schedule,
    )

    scored_data = score_data(
        model_input=train_mdl.outputs.model_output,
        test_data=test_input
    )
    scored_data.outputs.score_output.mode = "upload"

    eval_mdl = eval_model(
        scoring_result=scored_data.outputs.score_output
    )

    return {
        "trained_model": train_mdl.outputs.model_output,
        "scored_data": scored_data.outputs.score_output,
        "evaluation_report": eval_mdl.outputs.eval_output,
    }


pipeline = pipeline_with_components_from_yaml(
    training_input=Input(type="uri_folder", path="./resources/pipeline-from-yaml/data/"),
    test_input=Input(type="uri_folder", path="./resources/pipeline-from-yaml/data/"),
    training_max_epochs=20,
    training_learning_rate=1.8,
    learning_rate_schedule="time-based",
)
pipeline.settings.default_compute = AZURE_AML_CLUSTER
experiment_name = "expr-pipeline-from-yaml"
print(f"Created Pipeline: {experiment_name}")

job = ml_client.jobs.create_or_update(pipeline, experiment_name=experiment_name)
print(f"Submitted Job: {job.display_name}")

ml_client.jobs.stream(job.name)

결과는 AML Portal 에서 확인 가능하다. 이동해보자.